In [ ]:
!pip install lac
!pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 MB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
  Created wheel for lac: filename=LAC-2.1.2-py2.py3-none-any.whl size=64814699 sha256=e85df208a54c48d12ea39dda0c58aabf8eef103fb4d168787c61755a96a92bad
  Stored in directory: /root/.cache/pip/wheels/5b/a0/02/8fac032f2babbef354a8911bb7ad903f9cfc849458fdeab295
Successfully built lac
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 8.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# # 2. Use gdown tool to download from Google Drive link
!pip install gdown
import gdown
url = 'https://drive.google.com/u/0/uc?id=1VAzteetSSc9WOCne_u6-5oFt_6rIMR5E&export=download'
output = '/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx'
gdown.download(url, output, quiet=False)

Mounted at /content/drive


Downloading...
From: https://drive.google.com/u/0/uc?id=1VAzteetSSc9WOCne_u6-5oFt_6rIMR5E&export=download
To: /content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx
100%|██████████| 557M/557M [00:05<00:00, 93.9MB/s]


'/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# loading Data
news_data = pd.read_excel('/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/news_data.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
news_data.head()

,NewsID,Title,NewsContent,NewsSource
0,1,建设银行原董事长张恩照一审被判15年,本报记者 田雨 李京华 中国建设银行股份有限公司原董事长张恩照受贿案３日一审宣...,中国证券报
1,2,农行信用卡中心搬到上海滩,中国农业银行信用卡中心由北京搬到上海了！ 农行行长杨明生日前在信用卡中心揭牌仪式上...,人民日报
2,3,外运发展：价值型蓝筹股补涨要求强烈,在新基金快速发行以及申购资金回流的情况下，市场总体上呈现资金流动性过剩格局，考虑到现阶段...,杭州新希望
3,4,胜利股份：稳步走强形成标准上升通道,胜利股份（000407）公司子公司填海造地2800亩，以青岛的地价估算，静态价值在10亿...,源达投资
4,5,[港股快讯]恒指收市报18960点 成交467亿港元,全景网11月30日讯 外围股市造好，带动港股今早造好，恒指高开后反覆上升，最高升252点...,全景网


In [ ]:
news_data.shape

(1037035, 4)

In [ ]:
import re
import pandas as pd
import json
from rapidfuzz import process, fuzz

# Preliminary
json_path = '/content/drive/MyDrive/DSAA5002PROJECT/dsaa5002project/dsaa5002_project/A_share_list.json'
news_data['Title'] = news_data['Title'].astype(str)
news_data['NewsContent'] = news_data['NewsContent'].astype(str)
df = news_data[840000:]

# Read json file
with open(json_path, 'r', encoding='utf-8') as f:
    stock_list = json.load(f)

abbreviation_to_name = {
    'ST亚星': '亚星化学',
    'SST丰华': '丰华股份',
    '中信': '中信证券',
    '中金': '中金公司',
    '中石油': '中国石油',
    '中石化': '中国石化',
    '中钢': '中钢国际',
    '招行': '招商银行',
    '工行': '工商银行',
    '交行': '交通银行',
    '中行': '中国银行',
    '建行': '建设银行',
    '国寿': '中国人寿',
    '平保': '中国太保',
    '南航': '南方航空',
    '海航': '海南航空',
    '万科': '万科A',
    '茅台': '贵州茅台',
    '海尔': '青岛海尔'
}

# Prepare mapping of stock names, full names and codes
stock_info = {}
for stock in stock_list:
    name = stock['name']
    fullname = re.sub('股份有限公司$', '', stock['fullname'])
    code = re.sub('\D', '', stock['code'])
    stock_info[name] = name
    stock_info[fullname] = name
    stock_info[code] = name

for abbreviation, name in abbreviation_to_name.items():
    stock_info[abbreviation] = name

from LAC import LAC

lac = LAC(mode='lac')

def extract_org_names(text):
    lac_result = lac.run(text)
    words = lac_result[0]
    tags = lac_result[1]

    org_entities = [word for word, tag in zip(words, tags) if tag == 'ORG']
    unique_org_entities = list(set(org_entities))
    return unique_org_entities

def find_stock_names(org_names, stock_names, threshold=90):
    matched_names = set()
    for org_name in org_names:
        if org_name in ['中国人民银行', '中国央行', '央行', '中金所', '中国金融期货交易所', '公司', '京东', '美团']:
            continue
        else:
            best_match = process.extractOne(org_name, stock_names, scorer=fuzz.QRatio, score_cutoff=threshold)
            if best_match:
                if set(best_match[0]).issubset(set(org_name)) or set(org_name).issubset(set(best_match[0])):
                    matched_names.add(best_match[0])
    return matched_names

with open(json_path, 'r', encoding='utf-8') as f:
    stock_list = json.load(f)
stock_names = [stock['name'] for stock in stock_list]

def extract_stock_names(text, stock_info, stock_names):
    if not isinstance(text, str):
        return ''

    org_names = extract_org_names(text)

    fuzzy_matched_names = find_stock_names(org_names, stock_names)

    text_no_stars = text.replace('*', '')

    direct_matched_names = set()
    for variant, name in stock_info.items():
        if variant in text_no_stars:
            direct_matched_names.add(name)

    matched_names = direct_matched_names.union(fuzzy_matched_names)

    return ','.join(matched_names)

df['Explicit_Company'] = df.apply(lambda row: extract_stock_names(row['Title'], stock_info, stock_names) + ',' + extract_stock_names(row['NewsContent'], stock_info, stock_names), axis=1)
df['Explicit_Company'] = df['Explicit_Company'].str.strip(',')

def remove_duplicate_names(names):
    unique_names = set(names.split(','))
    return ','.join(unique_names)

# Apply the function to the 'Explicit_Company' column to remove duplicates
df['Explicit_Company'] = df['Explicit_Company'].apply(remove_duplicate_names)

# Now you can proceed with filtering the DataFrame
filtered_df = df[df['Explicit_Company'].apply(lambda x: len(x) > 0)]

# Output the result
print(len(filtered_df))


<ipython-input-10-db48863d36ea>:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Explicit_Company'] = df.apply(lambda row: extract_stock_names(row['Title'], stock_info, stock_names) + ',' + extract_stock_names(row['NewsContent'], stock_info, stock_names), axis=1)
<ipython-input-10-db48863d36ea>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Explicit_Company'] = df['Explicit_Company'].str.strip(',')  # 移除多余的逗号


104639


<ipython-input-10-db48863d36ea>:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Explicit_Company'] = df['Explicit_Company'].apply(remove_duplicate_names)


In [ ]:
filtered_df

,NewsID,Title,NewsContent,NewsSource,Explicit_Company
840000,840001,司法部建立健全律师行业东中西部对口帮扶机制,记者27日从司法部获悉，司法部办公厅发布《关于建立健全律师行业东中西部对口帮扶机制的方案》（...,人民网,人民网
840001,840002,教育部实施“国优计划” 组织高水平高校培养高素质中小学教师人才,据教育部网站消息，教育部近日印发《教育部关于实施国家优秀中小学教师培养计划的意见》（以下简称...,人民网-教育频道,人民网
840011,840012,重庆钢铁今日涨停 北向资金卖出2740.19万元,证券时报e公司讯，重庆钢铁今日涨10.13%，成交额10.7亿元，换手率7.64%，盘后龙虎...,证券时报·e公司,"重庆钢铁,中金公司"
840012,840013,北向资金今日净买入37.47亿元 宁德时代、贵州茅台分别获净买入15.18亿元、7.8亿元,7月27日，北向资金今日净买入37.47亿元，宁德时代、贵州茅台、工业富联分别获净买入15....,界面新闻,"工业富联,宁德时代,贵州茅台"
840014,840015,北向资金今日净买入37.47亿元 宁德时代净买入额居首,北向资金今日净买入37.47亿元。宁德时代、贵州茅台、工业富联分别获净买入15.18亿元、7...,南方财经网,"工业富联,宁德时代,贵州茅台"
...,...,...,...,...,...
1037030,1037031,亿华通：公司电解槽相关产品目前还处于产品的研发及测试阶段 尚未实现批量销售,每经AI快讯，有投资者在投资者互动平台提问：请问公司目前有没有电解槽产能，规划情况能否详细介...,每日经济新闻,亿华通
1037031,1037032,依米康：接受中泰证券调研,依米康（SZ 300249，收盘价：10.38元）发布公告称，2023年10月12日，依米康...,每日经济新闻,"中泰证券,依米康"
1037032,1037033,天风证券给予中核科技买入评级 核电行业景气上行 公司有望乘风而起,天风证券10月13日发布研报称，给予中核科技（000777.SZ，最新价：13.03元）买入...,每日经济新闻,"中核科技,天风证券"
1037033,1037034,海特生物：公司在抗癌药CPT获批后 会考虑适时开展CPT在海外的临床并谋求上市,有投资者提问：抗癌药CPT获批后，公司是否应该按照股权协议继续收购沙东股权，适应症为MM的C...,界面新闻,海特生物


In [ ]:
Be_filtered_df = df[df['Explicit_Company'].apply(lambda x: len(x) == 0)]
Be_filtered_df.to_excel('/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/Flitered/Be_filtered1_part5.xlsx' ,encoding='utf-8')
filtered_df.to_excel('/content/drive/MyDrive/DSAA5002PROJECT/Task1_data/Flitered/Task1_part1-5.xlsx' ,encoding='utf-8')